<a href="https://colab.research.google.com/github/timhalter/PA-Addee/blob/main/Addee_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
import pandas as pd
import numpy as np


     |████████████████████████████████| 2.9 MB 11.3 MB/s 
     |████████████████████████████████| 596 kB 41.7 MB/s 
     |████████████████████████████████| 56 kB 5.1 MB/s 
     |████████████████████████████████| 3.3 MB 34.4 MB/s 
     |████████████████████████████████| 895 kB 28.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
url = 'https://raw.githubusercontent.com/timhalter/PA-Addee/main/children_books.csv'
df = pd.read_csv(url,encoding='ISO-8859-1')
df.drop(['Author', 'Inerest_age'], axis=1, inplace=True)
df.rename(columns = {'Title':'title', 'Desc':'desc', 'Reading_age':'age'}, inplace = True)

df.head()

,title,desc,age
0,The Girl Who Drank the Moon,"Every year, the evil Protectorate offers a bab...",10-14
1,Time Between Us,Sixteen-year-old Anna is struggling to underst...,12+
2,Girl Out of Water,Lou Brown's swimming ambitions sank without tr...,10+
3,Captive,Robyn is scared. Ever since the attempted assa...,13
4,The School of Music,Welcome to the School of Music. In charge is M...,10+


In [3]:
seq_len = 512
num_samples = len(df)

Xids = np.zeros((num_samples, seq_len))
Xmask = np.zeros((num_samples, seq_len))

Xids.shape

(3269, 512)

In [4]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

for i, description in enumerate(df['desc']):
      tokens = tokenizer.encode_plus(description, max_length=seq_len, truncation=True,
                                     padding='max_length', add_special_tokens=True,
                                     return_tensors='tf')
      Xids[i, :] = tokens['input_ids']
      Xmask[i, :] = tokens['attention_mask']

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [5]:
Xids

array([[  101.,  4081.,  1214., ...,     0.,     0.,     0.],
       [  101., 22177.,   118., ...,     0.,     0.,     0.],
       [  101., 10111.,  2671., ...,     0.,     0.,     0.],
       ...,
       [  101.,  2485.,  1113., ...,     0.,     0.,     0.],
       [  101., 22849.,  6748., ...,     0.,     0.,     0.],
       [  101.,  1109.,  2174., ...,     0.,     0.,     0.]])

In [6]:
# Xmask is for BERT to separate real data (1) from padding tokens (0)
Xmask

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]])

Functions to separate the ages

In [7]:
def get_min_age(sir):
    if sir.find('-') != -1:
        return int(sir.split('-')[0])
    else:
        return int(sir.replace('+',' ').split(' ')[0])

def get_max_age(sir):
    if sir.find('-') != -1:
        return int(sir.split('-')[1])
    elif sir.find('+') != -1:
        sp = sir.replace(' ', '')
        sp = sir.replace('+', '')
        return int(sp)
    else:
        return 99

Add separate age columns

In [8]:
df.names = df.title.str.lower()
df.desc = df.desc.str.lower()
df['Min_Age'] = df.age.apply(get_min_age)
df['Max_Age'] = df.age.apply(get_max_age)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [9]:
arr = df['Min_Age'].values
arr

array([10, 12, 10, ..., 10, 11, 12])

In [10]:
labels = np.zeros((num_samples, arr.max()+1))
labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [11]:
labels[np.arange(num_samples), arr] = 1
labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

Turn data readable for tensorflow

In [12]:
import tensorflow as tf

In [13]:
dataset = tf.data.Dataset.from_tensor_slices((Xids, Xmask, labels))
dataset.take(1)

<TakeDataset shapes: ((512,), (512,), (15,)), types: (tf.float64, tf.float64, tf.float64)>

Merge input tensors into single dictionary

In [14]:
def map_func(input_ids, masks, labels):
  return {'input_ids': input_ids, 'attention_mask': masks}, labels

In [15]:
dataset = dataset.map(map_func)

In [16]:
dataset.take(1)

<TakeDataset shapes: ({input_ids: (512,), attention_mask: (512,)}, (15,)), types: ({input_ids: tf.float64, attention_mask: tf.float64}, tf.float64)>

In [17]:
batch_size = 16

dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True)
dataset.take(1)

<TakeDataset shapes: ({input_ids: (16, 512), attention_mask: (16, 512)}, (16, 15)), types: ({input_ids: tf.float64, attention_mask: tf.float64}, tf.float64)>

In [18]:
split = 0.9

size = int((num_samples / batch_size) * split)

In [19]:
# new number of samples within our data
size

183

Train/Test split

In [20]:
train_ds = dataset.take(size)
val_ds = dataset.skip(size)

del dataset

In [21]:
from transformers import TFAutoModel

bert = TFAutoModel.from_pretrained('bert-base-cased')

bert.summary()

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "tf_bert_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  108310272 
Total params: 108,310,272
Trainable params: 108,310,272
Non-trainable params: 0
_________________________________________________________________


In [22]:
 input_ids = tf.keras.layers.Input(shape=(seq_len,), name='input_ids', dtype='int32')
 mask = tf.keras.layers.Input(shape=(seq_len,), name='attention_mask', dtype='int32')

 embeddings = bert.bert(input_ids, attention_mask=mask)[1]

# convert embeddings into label predictions
 x = tf.keras.layers.Dense(1024, activation='relu')(embeddings)
 y = tf.keras.layers.Dense(arr.max()+1, activation='softmax', name='outputs')(x)

In [31]:
model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

# (optional) freeze bert layer
model.layers[2].trainable = False
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 512)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 108310272   input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
dense (Dense)                   (None, 1024)         787456      bert[0][1]                 

In [32]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-6)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [34]:
model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

Train the model

In [37]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=3
)

Epoch 1/3
 12/183 [>.............................] - ETA: 3:34 - loss: 1.7347 - accuracy: 0.3021

KeyboardInterrupt: ignored

In [36]:
model.save('Age_model')

INFO:tensorflow:Assets written to: Age_model/assets


INFO:tensorflow:Assets written to: Age_model/assets
/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
